# 🏆 06. 실전 자동화 프로젝트 — 차트 생성 & 스케줄링

> **목표**: xlwings로 Excel 차트를 자동 생성하고, **매일 자동 실행**되는 자동화 스크립트를 만든다.
>
> **왜 중요한가?** — "매일 아침마다 보고서 만들어서 차트 넣어줘"라는 요청. 직접 하면 30분, 자동화하면 **0분(자동)**.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | 차트 기초 생성 | 데이터 범위 → 차트 자동 삽입 |
| 2 | 차트 종류 & 스타일 | 막대/꺾은선/원형 선택법 |
| 3 | 차트 세부 설정 | 제목, 범례, 데이터 레이블 |
| 4 | schedule 스케줄링 | 매일/매주 자동 실행 |
| 5 | 🔥 실전 미니 프로젝트 | 데이터 수집 → 차트 → 자동 저장 |

---
## 0️⃣ 준비: 라이브러리 설치 & 파일 열기

```bash
pip install xlwings schedule
```

In [ ]:
import xlwings as xw

app = xw.App(add_book=False)
wb = app.books.add()           # 새 워크북으로 실습
ws = wb.sheets[0]
ws.name = '월별매출'

print(f'✅ 엑셀 실행 완료 | 시트: {ws.name}')

In [ ]:
# 실습용 샘플 데이터 입력
data = [
    ['월',   '매출',  '비용',  '순이익'],
    ['1월',  1200,   800,    400],
    ['2월',  1500,   900,    600],
    ['3월',  1800,   1000,   800],
    ['4월',  2100,   1100,   1000],
    ['5월',  1900,   950,    950],
    ['6월',  2300,   1200,   1100],
]

ws.range('A1').value = data

# 헤더 서식
ws.range('A1:D1').font.bold = True
ws.range('A1:D1').color = (68, 114, 196)
ws.range('A1:D1').font.color = (255, 255, 255)
ws.range('A1:D1').columns.autofit()

print('✅ 샘플 데이터 입력 완료')
print('📊 A1:D7 범위에 月별 매출/비용/순이익 데이터가 입력됐습니다')

---
## 1️⃣ 차트 기초 생성

> `ws.charts.add(left, top, width, height)` — 차트 객체를 워크시트에 추가합니다.

In [ ]:
# 차트 추가 (위치: left=0, top=120 / 크기: 500x280)
chart = ws.charts.add(left=10, top=130, width=500, height=280)

# 데이터 범위 연결 (A1:D7 = 헤더+데이터 전체)
chart.set_source_data(ws.range('A1:D7'))

# 차트 이름 지정 (나중에 다시 찾을 때 사용)
chart.name = 'monthly_chart'

print(f'✅ 차트 생성 완료')
print(f'📍 차트 이름: {chart.name}')
print(f'📐 크기: {chart.width} x {chart.height}')

---
## 2️⃣ 차트 종류 변경

| 차트 종류 | `chart_type` 값 | API 상수 | 용도 |
|:---|:---:|:---:|:---|
| 묶은 세로 막대 | `'bar_clustered'` | `51` | 항목 비교 |
| 꺾은선 | `'line'` | `-4100` | 추세 파악 |
| 원형(파이) | `'pie'` | `5` | 비율 표현 |
| 가로 막대 | `'bar'` | `-4099` | 긴 항목명 |
| 영역형 | `'area'` | `1` | 누적 면적 |

> 💡 xlwings의 `chart_type` 문자열 방식이 가장 간단합니다!

In [ ]:
# 차트 가져오기
chart = ws.charts['monthly_chart']

# 꺾은선 그래프로 변경
chart.chart_type = 'line'
print('✅ 꺾은선 그래프로 변경')

In [ ]:
# 묶은 세로 막대 그래프로 변경
chart.chart_type = 'bar_clustered'
print('✅ 세로 막대 그래프로 변경')

---
## 3️⃣ 차트 세부 설정 (제목, 범례, 레이블)

> 세부 설정은 `.api[1]`로 COM 객체에 접근해야 합니다.

In [ ]:
chart = ws.charts['monthly_chart']
c = chart.api[1]   # COM 객체 접근

# ── 제목 설정 ──
c.HasTitle = True
c.ChartTitle.Text = '📊 월별 매출 현황'
c.ChartTitle.Font.Size = 14
c.ChartTitle.Font.Bold = True

# ── 범례 위치 설정 ──
# 위치 상수: 1=위, 2=오른쪽, 3=왼쪽, 4=아래
c.Legend.Position = -4107   # xlBottom (아래)

# ── 축 제목 ──
c.Axes(2).HasTitle = True           # 2 = Y축
c.Axes(2).AxisTitle.Text = '금액(만원)'

print('✅ 차트 제목 / 범례 / 축 제목 설정 완료')

In [ ]:
# ── 데이터 레이블 추가 (첫 번째 계열 = 매출) ──
series1 = c.SeriesCollection(1)   # 1번 계열 = 매출
series1.HasDataLabels = True
series1.DataLabels().NumberFormat = '#,##0'   # 천 단위 구분

print('✅ 매출 계열에 데이터 레이블 추가 완료')
print('💡 엑셀에서 막대 위에 숫자가 표시되는 걸 확인해보세요!')

---
## 4️⃣ 스케줄링 — 매일 자동 실행

Python 스크립트를 **특정 시간에 자동으로 반복 실행**하는 방법 3가지

| 방법 | 장점 | 단점 |
|:---|:---|:---|
| `schedule` 라이브러리 | 코드가 간단, 유연 | PC가 켜져있고 파이썬이 실행 중이어야 함 |
| Windows 작업 스케줄러 | 안정적, 부팅 후 자동 실행 가능 | Windows 전용, 설정이 약간 복잡 |
| `APScheduler` | 고급 기능 (cron 등) | 추가 설치 필요 |

In [ ]:
# schedule 설치 확인
try:
    import schedule
    print(f'✅ schedule 설치됨 (버전: {schedule.__version__})')
except ImportError:
    print('❌ schedule 미설치 → 아래 명령 실행:')
    print('   pip install schedule')

In [ ]:
import schedule
import time
from datetime import datetime

# ─── 실행할 함수 정의 ───
def my_job():
    """매일 자동 실행될 작업"""
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'[{now}] ✅ 자동 작업 실행!')
    # 여기에 엑셀 자동화 코드 넣으면 됩니다

# ─── 스케줄 등록 ───
schedule.every().day.at('09:00').do(my_job)      # 매일 09:00
schedule.every().monday.at('08:30').do(my_job)   # 매주 월요일 08:30
schedule.every(10).minutes.do(my_job)            # 10분마다
schedule.every().hour.do(my_job)                 # 매시간

print('✅ 스케줄 구문 예시 - 실제 실행은 아래 셀에서!') 
print('💡 schedule.clear() 로 등록된 스케줄을 모두 삭제할 수 있습니다')

In [ ]:
import schedule
import time
from datetime import datetime

# ─── 테스트: 3초마다 실행 (10초 후 자동 종료) ───
schedule.clear()  # 이전 스케줄 초기화

run_count = 0
max_runs = 3

def test_job():
    global run_count
    run_count += 1
    now = datetime.now().strftime('%H:%M:%S')
    print(f'  [{now}] 🔔 실행 #{run_count}')

schedule.every(3).seconds.do(test_job)

print(f'⏰ 3초마다 {max_runs}회 실행합니다...')
while run_count < max_runs:
    schedule.run_pending()
    time.sleep(1)

schedule.clear()
print('\n✅ 테스트 완료!')

---
## 5️⃣ Windows 작업 스케줄러 연동 (실무 추천 ⭐)

> Python 스크립트를 `.py` 파일로 저장 후 Windows 작업 스케줄러에 등록하면,  
> **PC 부팅 후 자동으로 실행**됩니다 (Jupyter 없이도 가능!).

### 방법

**1단계**: 아래 코드를 `run_report.py`로 저장

```python
# run_report.py
import xlwings as xw
from datetime import date
import random, os

def create_report():
    FILE = r'E:\excelAuto\report.xlsx'
    app = xw.App(visible=False)
    try:
        wb = app.books.open(FILE) if os.path.exists(FILE) else app.books.add()
        ws = wb.sheets[0]
        
        # A열에 날짜, B열에 값 추가
        last_row = ws.range('A1').expand('down').last_cell.row + 1
        ws.range(f'A{last_row}').value = str(date.today())
        ws.range(f'B{last_row}').value = random.randint(1000, 5000)
        
        wb.save(FILE)
    finally:
        app.quit()

create_report()
print('완료')
```

**2단계**: CMD에서 등록

```bash
# 매일 오전 9시 자동 실행
schtasks /create /tn "엑셀자동화" /tr "python E:\excelAuto\run_report.py" /sc daily /st 09:00

# 등록된 작업 확인
schtasks /query /tn "엑셀자동화"

# 지금 즉시 실행 (테스트)
schtasks /run /tn "엑셀자동화"

# 삭제
schtasks /delete /tn "엑셀자동화" /f
```

---
## 🔥 6. 실전 미니 프로젝트

**시나리오**: 매일 실행 → 오늘 데이터 추가 → 꺾은선 차트 자동 갱신 → 파일 저장

```
실행 흐름
────────
[오전 9시 자동 실행]
    │
    ▼
파일 열기 (없으면 신규 생성)
    │
    ▼
오늘 날짜 + 데이터 마지막 행에 추가
    │
    ▼
차트 시트 갱신 (기존 차트 삭제 후 재생성)
    │
    ▼
파일 저장 → 완료
```

In [ ]:
import xlwings as xw
import random
from datetime import date
from pathlib import Path

FILE_PATH = str(Path('.') / 'daily_report.xlsx')

def create_daily_report():
    """매일 자동 실행: 데이터 추가 → 차트 갱신 → 저장"""
    
    app_r = xw.App(add_book=False, visible=True)
    
    try:
        # ─── 파일 열기 / 신규 생성 ───────────────────
        if Path(FILE_PATH).exists():
            wb_r = app_r.books.open(FILE_PATH)
            print(f'📂 기존 파일 열기: {FILE_PATH}')
        else:
            wb_r = app_r.books.add()
            print(f'📄 신규 파일 생성')
        
        # ─── 데이터 시트 ─────────────────────────────
        sheet_names = [s.name for s in wb_r.sheets]
        
        if '데이터' not in sheet_names:
            ws_data = wb_r.sheets.add('데이터')
            ws_data.range('A1').value = ['날짜', '매출', '방문자']
            ws_data.range('A1:C1').font.bold = True
            ws_data.range('A1:C1').color = (68, 114, 196)
            ws_data.range('A1:C1').font.color = (255, 255, 255)
            print('  ✅ 데이터 시트 생성')
        else:
            ws_data = wb_r.sheets['데이터']
        
        # ─── 오늘 데이터 추가 ─────────────────────────
        last_row = ws_data.range('A1').expand('down').last_cell.row + 1
        ws_data.range(f'A{last_row}').value = [
            date.today().strftime('%m/%d'),
            random.randint(800, 2500),   # 매출 (실습용 랜덤)
            random.randint(100, 600)     # 방문자
        ]
        print(f'  ✅ {last_row}행: {date.today()} 데이터 추가')
        
        # ─── 차트 시트 갱신 ──────────────────────────
        if '차트' in sheet_names:
            wb_r.sheets['차트'].delete()
        
        ws_chart = wb_r.sheets.add('차트', after=ws_data)
        
        # 데이터 복사
        data_all = ws_data.range('A1').expand('table').value
        ws_chart.range('A1').value = data_all
        ws_chart.range('A1:C1').font.bold = True
        ws_chart.range('A1:C1').color = (68, 114, 196)
        ws_chart.range('A1:C1').font.color = (255, 255, 255)
        ws_chart.range('A1').columns.autofit()
        
        # 차트 범위
        total_rows = len(data_all)
        data_range = ws_chart.range(f'A1:B{total_rows}')  # 날짜 + 매출만
        
        # 차트 생성
        ch = ws_chart.charts.add(left=10, top=160, width=520, height=300)
        ch.set_source_data(data_range)
        ch.chart_type = 'line'
        ch.name = 'trend_chart'
        
        # 차트 제목
        c = ch.api[1]
        c.HasTitle = True
        c.ChartTitle.Text = f'📈 일별 매출 추이 (마지막 업데이트: {date.today()})'
        c.ChartTitle.Font.Size = 12
        c.Axes(2).HasTitle = True
        c.Axes(2).AxisTitle.Text = '매출'
        
        print(f'  ✅ 차트 생성 완료 (데이터 {total_rows-1}건)')
        
        # ─── 저장 ────────────────────────────────────
        wb_r.save(FILE_PATH)
        print(f'\n💾 저장 완료: {FILE_PATH}')
        print(f'🎉 리포트 업데이트 완료!')
        
    except Exception as e:
        print(f'\n❌ 에러 발생: {e}')
        raise
    finally:
        # 확인 후 닫기: True로 바꾸면 자동 종료
        AUTO_CLOSE = False
        if AUTO_CLOSE:
            app_r.quit()


print('✅ create_daily_report() 정의 완료')
print('💡 아래 셀에서 실행해보세요!')

In [ ]:
# 즉시 실행!
create_daily_report()
# → daily_report.xlsx 파일이 생성되고 차트가 그려집니다

In [ ]:
# 한 번 더 실행하면 데이터가 쌓이는 걸 확인할 수 있어요
create_daily_report()

print('\n💡 여러 번 실행할수록 차트 꺾은선이 길어집니다!')

---
## 💾 마무리

In [ ]:
# 엑셀 닫기 (app 변수가 있는 경우)
# app.quit()

print('💡 엑셀 확인 후 app.quit() 실행하세요')

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `chart.api[1]` AttributeError | COM 객체 접근 실패 | `[1]` 인덱스 확인, xlwings 최신 버전 |
| 차트가 데이터 반영 안 됨 | `set_source_data()` 범위 오류 | 범위 주소 직접 확인 후 재지정 |
| `schedule` 가 실행 안 됨 | `run_pending()` 루프 없음 | `while True: schedule.run_pending(); time.sleep(1)` |
| 작업 스케줄러에서 Python 못 찾음 | 경로 문제 | `where python` 으로 전체 경로 확인 후 등록 |
| 파일 저장 실패 | 이미 열려있는 파일 | 다른 xlsx 프로세스 종료 후 재시도 |

---
## 📝 연습 문제

1. 차트 종류를 `'bar_clustered'`로 바꿔보세요
2. `create_daily_report()`에 **비용** 계열을 추가해보세요 (C열 = 방문자 → 비용으로 수정)
3. Windows 작업 스케줄러에 `schtasks` 명령으로 등록해보세요

In [ ]:
# 🏋️ 연습: 원형 차트(파이 차트) 만들기
# 힌트: chart_type = 'pie'  &  데이터는 1개 계열만 선택

# 여기에 코드 작성!


---
## 📌 다음 챕터 예고

> **07. pandas 심화 — 데이터 분석 & Excel 입출력**  
> pandas로 대용량 데이터를 분석하고, 피벗/그룹바이/조건 필터링 결과를 Excel로 깔끔하게 출력하는 방법